In [3]:
import os
import torch

import csv 

import pandas as pd

In [11]:
data = []
names = []
target = []

data_sheet = pd.read_csv("CXR14/CXR8/Data_Entry_2017_v2020.csv")

print(data_sheet)

N = data_sheet.shape[0]

             Image Index          Finding Labels  Follow-up #  Patient ID  \
0       00000001_000.png            Cardiomegaly            0           1   
1       00000001_001.png  Cardiomegaly|Emphysema            1           1   
2       00000001_002.png   Cardiomegaly|Effusion            2           1   
3       00000002_000.png              No Finding            0           2   
4       00000003_001.png                  Hernia            0           3   
...                  ...                     ...          ...         ...   
112115  00030801_001.png          Mass|Pneumonia            1       30801   
112116  00030802_000.png              No Finding            0       30802   
112117  00030803_000.png              No Finding            0       30803   
112118  00030804_000.png              No Finding            0       30804   
112119  00030805_000.png              No Finding            0       30805   

        Patient Age Patient Gender View Position  OriginalImage[Width  \
0 

In [ ]:
from imageio import imread 
import PIL 
from PIL import Image 
from torchvision import transforms 

import torch
from torchvision import transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


model = torch.hub.load('pytorch/vision:v0.5.0', 'densenet121', pretrained=True).to(device)
# or any of these variants
#model = torch.hub.load('pytorch/vision:v0.5.0', 'densenet169', pretrained=True).to(device)
#model = torch.hub.load('pytorch/vision:v0.5.0', 'densenet201', pretrained=True).to(device)
#model = torch.hub.load('pytorch/vision:v0.5.0', 'densenet161', pretrained=True).to(device)
model.eval()

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def run_through_denseNetPyTorch(img, densenet_model, device):
    
    img = torch.unsqueeze(img.to(device), 0)
    
    with torch.no_grad():
        features = densenet_model.features( img )
        f_relu = torch.nn.functional.relu( features )
        pool = torch.nn.functional.adaptive_avg_pool2d( f_relu, (1,1) )

        y_st = pool.cpu().numpy().reshape(-1)
    
    return y_st


X_train_0 = [] 
y_train_0 = []

fnames = data_sheet['Image Index']
labels = data_sheet['View Position']

for i in range(N):
    if i%10000 == 0:
        print('Completed ', i, ' of ', N)
    
    
    filename = 'CXR14/CXR8/images/extracted/'+fnames[i]
    img = imread(filename, as_gray=False, pilmode="RGB") 
    
    img = Image.fromarray(img)
    img = PIL.ImageOps.equalize(img)
    img = preprocess(img)
    
    X_train_0.append(run_through_denseNetPyTorch(img=img, densenet_model=model, device=device))
    if labels[i] == 'PA':
        y_train_0.append(0)
    elif labels[i] == 'AP':
        y_train_0.append(1)
    else:
        print('View Position not PA or AP. Instead found ', labels[i], 'at index', i)
        print(wronginfor)
    

In [18]:
import numpy as np

X_train_0 = np.array(X_train_0)
y_train_0 = np.array(y_train_0)
fnames = np.array(fnames)
print(X_train_0.shape)
print(np.sum(y_train_0==0))

(112120, 1024)
67310


In [20]:
with open("CXR14_DenseNet.npy", 'wb') as f:
    np.save(f,X_train_0)
    np.save(f,y_train_0)
    np.save(f,fnames)